In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'project-mimic-430923'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [4]:
# 1. Get basic patient info (admissions + demographics (excluding marital_status) + death status)
patient_info_query = """
SELECT adm.subject_id, adm.hadm_id, adm.admittime, adm.dischtime, pat.gender, pat.anchor_age AS age, adm.race, adm.hospital_expire_flag
FROM `physionet-data.mimiciv_hosp.admissions` AS adm
JOIN `physionet-data.mimiciv_hosp.patients` AS pat
ON adm.subject_id = pat.subject_id
WHERE adm.admittime IS NOT NULL
ORDER BY subject_id
"""
patient_info_df = run_query(patient_info_query)

patient_info_df['in_hospital_death'] = patient_info_df['hospital_expire_flag'] == 1

patient_info_df = patient_info_df.drop(columns=['hospital_expire_flag'])

# Calculate length of stay and have data only with positive L.O.F
patient_info_df['admittime'] = pd.to_datetime(patient_info_df['admittime'])
patient_info_df['dischtime'] = pd.to_datetime(patient_info_df['dischtime'])
patient_info_df['length_of_stay'] = (patient_info_df['dischtime'] - patient_info_df['admittime']).dt.total_seconds() / (60 * 60 * 24)

patient_info_df = patient_info_df[patient_info_df['length_of_stay'] > 0]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [5]:
patient_info_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
0,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,F,52,WHITE,False,1.015278
1,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,F,52,WHITE,False,2.222222
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,F,52,WHITE,False,1.754167
3,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,F,52,WHITE,False,0.786111
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,F,19,WHITE,False,0.298611
...,...,...,...,...,...,...,...,...,...
431226,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,F,46,WHITE,False,17.074306
431227,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,F,46,WHITE,False,10.011111
431228,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,M,58,WHITE,False,3.491667
431229,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58,WHITE,True,6.996528


In [6]:
# Count number of patients
num_patients = patient_info_df['subject_id'].nunique()
print(f"Number of patients: {num_patients}")

Number of patients: 180677


In [7]:
# 2. Get item IDs for lipase tests
lipase_item_query = """
SELECT itemid, label, fluid
FROM `physionet-data.mimiciv_hosp.d_labitems`
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df = run_query(lipase_item_query)
print(lipase_items_df)
lipase_itemids = [50956, 50844, 51055, 51036] # blood and other fluid # Lipase item IDs [50956]# only blood

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   itemid               label             fluid
0   50956              Lipase             Blood
1   50844     Lipase, Ascites           Ascites
2   51055     Lipase, Pleural           Pleural
3   51036  Lipase, Body Fluid  Other Body Fluid


In [8]:
# 3. Retrieve lipase values
lipase_values_query = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid IN ({', '.join(map(str, lipase_itemids))})
ORDER BY subject_id, charttime
"""
lipase_values_df = run_query(lipase_values_query)

# Merge lipase values with patient info and clean data
lipase_values_df = pd.merge(lipase_values_df, patient_info_df, on=['subject_id', 'hadm_id'], how='left')
lipase_values_df.dropna(subset=['age', 'lipase_level'], inplace=True)

# Identify patients with high lipase levels
def check_lipase(row):
    upper_limit = 140 if row['age'] < 60 else 151
    return row['lipase_level'] >= 3 * upper_limit

high_lipase_df = lipase_values_df[lipase_values_df.apply(check_lipase, axis=1)]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [9]:
lipase_values_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
4,10000084,23052089,2160-11-20 22:30:00,47.0,2160-11-21 01:56:00,2160-11-25 14:52:00,M,72,WHITE,False,4.538889
15,10000826,20032235,2146-12-06 04:24:00,33.0,2146-12-05 19:07:00,2146-12-12 16:30:00,F,32,WHITE,False,6.890972
16,10000826,28289260,2146-12-30 17:30:00,51.0,2146-12-31 00:43:00,2147-01-02 17:45:00,F,32,WHITE,False,2.709722
20,10001176,23334588,2186-11-28 22:00:00,21.0,2186-11-29 03:56:00,2186-12-02 15:00:00,F,64,WHITE,False,3.461111
25,10001338,22119639,2138-05-10 05:25:00,17.0,2138-05-09 19:47:00,2138-05-27 15:40:00,F,43,WHITE,False,17.828472
...,...,...,...,...,...,...,...,...,...,...,...
229741,19999303,23567530,2161-04-03 19:50:00,292.0,2161-04-03 15:40:00,2161-04-06 10:45:00,F,61,WHITE,False,2.795139
229742,19999303,23567530,2161-04-04 05:10:00,276.0,2161-04-03 15:40:00,2161-04-06 10:45:00,F,61,WHITE,False,2.795139
229752,19999828,29734428,2147-07-17 18:04:00,23.0,2147-07-18 16:23:00,2147-08-04 18:10:00,F,46,WHITE,False,17.074306
229754,19999840,26071774,2164-07-25 06:45:00,19.0,2164-07-25 00:27:00,2164-07-28 12:15:00,M,58,WHITE,False,3.491667


In [10]:
# 2. Get item IDs for lipase tests in icu
lipase_item_query_icu = """
SELECT itemid, label, category
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df_icu = run_query(lipase_item_query_icu)
print(lipase_items_df_icu)
lipase_itemids_icu = "225672"  # Lipase item IDs

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   itemid   label category
0  225672  Lipase     Labs


In [11]:
# 3. Retrieve lipase values icu

lipase_values_query_icu = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN ({lipase_itemids_icu})
ORDER BY subject_id, charttime
"""
lipase_values_df_icu = run_query(lipase_values_query_icu)

# Merge lipase values with patient info and clean data
lipase_values_df_icu = pd.merge(lipase_values_df_icu, patient_info_df, on=['subject_id', 'hadm_id'], how='left')
lipase_values_df_icu.dropna(subset=['age', 'lipase_level'], inplace=True)

# Identify patients with high lipase levels in ICU
def check_lipase(row):
    upper_limit = 140 if row['age'] < 60 else 151
    return row['lipase_level'] >= 3 * upper_limit

high_lipase_df_icu = lipase_values_df_icu[lipase_values_df_icu.apply(check_lipase, axis=1)]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [12]:
high_lipase_df_icu

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
1,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556
15,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944
16,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944
55,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528
56,10036086,28728587,2196-05-28 03:39:00,777.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528
...,...,...,...,...,...,...,...,...,...,...,...
17140,19882958,29628147,2182-08-31 03:10:00,787.0,2182-08-29 17:25:00,2182-09-03 13:50:00,M,83,WHITE,False,4.850694
17167,19899716,21665899,2143-09-03 17:16:00,587.0,2143-09-03 12:59:00,2143-09-08 18:00:00,M,62,BLACK/AFRICAN AMERICAN,False,5.209028
17178,19901341,23906609,2169-08-10 06:48:00,588.0,2169-08-06 14:52:00,2169-09-05 14:58:00,F,55,WHITE,False,30.004167
17189,19907884,24707264,2181-01-24 04:42:00,789.0,2181-01-23 21:32:00,2181-02-03 13:30:00,F,38,WHITE,False,10.665278


In [13]:
high_lipase_df_combined = pd.concat([high_lipase_df, high_lipase_df_icu])

# Drop duplicate rows
high_lipase_df_unique = high_lipase_df_combined.drop_duplicates()

In [14]:
len(high_lipase_df_unique)

4241

In [15]:
# Count number of patients with lipase levels recorded
num_patients = high_lipase_df_unique['subject_id'].nunique()
print(f"Number of unique patients with high lipase levels: {num_patients}")

Number of unique patients with high lipase levels: 2106


In [16]:
ap_icd_query = f"""
SELECT subject_id, hadm_id, icd_code
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'K85%' or icd_code = '5770'
ORDER BY subject_id
"""
ap_icd_df = run_query(ap_icd_query)

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [17]:
ap_icd_df

,subject_id,hadm_id,icd_code
0,10002807,28464737,K8590
1,10004606,29242151,K8510
2,10006431,24638489,K8580
3,10007795,25135483,5770
4,10007795,28477357,5770
...,...,...,...
5889,19990545,23106222,K8590
5890,19993764,23707485,K851
5891,19996968,29843339,5770
5892,19996968,28227793,5770


In [18]:
ap_icd_df['subject_id'].nunique()

3850

In [19]:
merged_ap_icd_lp_df = pd.merge(high_lipase_df_unique, ap_icd_df, on=['subject_id', 'hadm_id'], how='inner')

# Merge length of stay into the dataset
merged_ap_icd_lp_df = pd.merge(merged_ap_icd_lp_df, patient_info_df[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='left')

In [20]:
merged_ap_icd_lp_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,icd_code
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167,K8580
2,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,5770
3,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,5770
4,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False,8.009028,5770
...,...,...,...,...,...,...,...,...,...,...,...,...
2800,11741025,28221700,2135-04-29 17:20:00,2872.0,2135-04-29 12:52:00,2135-05-02 14:40:00,M,91,WHITE,False,3.075000,K851
2801,13990946,21605620,2129-01-16 21:38:00,616.0,2129-01-16 21:33:00,2129-01-23 15:34:00,M,47,OTHER,False,6.750694,K8581
2802,15542216,21588047,2158-07-24 04:23:00,2683.0,2158-07-24 03:49:00,2158-07-27 18:42:00,F,56,HISPANIC/LATINO - PUERTO RICAN,False,3.620139,5770
2803,17417573,26862398,2202-08-24 22:23:00,2318.0,2202-08-24 21:33:00,2202-09-04 16:10:00,M,41,WHITE,False,10.775694,5770


In [21]:
# Count the number of patients with high lipase levels who had a CT scan
num_ap_icd_lp_df = merged_ap_icd_lp_df['subject_id'].nunique()
print(f"Number of patients with high lipase levels and were diagnosed with AP: {num_ap_icd_lp_df}")

# Count the number of rows in the dataset
num_rows = merged_ap_icd_lp_df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of patients with high lipase levels and were diagnosed with AP: 1366
Number of rows in the dataset: 2805


In [22]:
def select_row(group):
    # Check if there's any row with hospital_expire_flag = 1
    if (group['in_hospital_death']).any():
        # Return the first row with hospital_expire_flag = 1
        return group[group['in_hospital_death']].iloc[0]
    else:
        # Otherwise, return the row with the maximum length_of_stay
        return group.loc[group['length_of_stay'].idxmax()]

# Apply the function to each group and reset the index
ap_lp_filtered_df = merged_ap_icd_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-22-2c7659cf5a91>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ap_lp_filtered_df = merged_ap_icd_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [23]:
ap_lp_filtered_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,icd_code
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167,K8580
2,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,5770
3,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False,8.009028,5770
4,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528,5770
...,...,...,...,...,...,...,...,...,...,...,...,...
1361,19951256,20952163,2149-09-02 21:26:00,3241.0,2149-09-03 00:14:00,2149-09-04 17:45:00,F,56,HISPANIC/LATINO - COLUMBIAN,False,1.729861,5770
1362,19970491,22119205,2131-02-11 03:06:00,487.0,2131-02-10 18:25:00,2131-02-17 14:15:00,M,55,WHITE,False,6.826389,5770
1363,19977727,23685838,2170-06-12 07:20:00,528.0,2170-06-10 22:54:00,2170-06-12 14:37:00,F,64,WHITE,False,1.654861,5770
1364,19990545,23106222,2139-10-06 08:30:00,1886.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,24.690278,K8590


In [ ]:
ap_lp_filtered_df.to_csv('AP_ICD_Lipase_Dataset.csv', encoding='utf-8', index = False)

In [24]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in ap_lp_filtered_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(ap_lp_filtered_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
2    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
3    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
4    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   

            dischtime gender  age   race  in_hospital_death  length_of_stay  \
0 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
1 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
2 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
3 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
4 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   

  icd_code_x icd_code_y  seq_num  icd_

In [25]:
merged_comorbidities_df = merged_comorbidities_df.rename(columns={"icd_code_x": "icd_code_AP", "icd_code_y": "icd_code"})

In [26]:
merged_comorbidities_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,icd_code_AP,icd_code,seq_num,icd_version
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,G40409,1,10
1,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,K8510,2,10
2,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,G9340,3,10
3,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,K8064,4,10
4,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,E871,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18998,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,5770,1,9
18999,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,30391,2,9
19000,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,4019,3,9
19001,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,V1581,4,9


In [27]:
icd_title_query = """
SELECT icd_code, icd_version, long_title
FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
"""
icd_titles_df = run_query(icd_title_query)

merged_comorbidities_df = pd.merge(merged_comorbidities_df, icd_titles_df, on=['icd_code', 'icd_version'], how='left')

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [28]:
merged_comorbidities_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,icd_code_AP,icd_code,seq_num,icd_version,long_title
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,G40409,1,10,Other generalized epilepsy and epileptic syndr...
1,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,K8510,2,10,Biliary acute pancreatitis without necrosis or...
2,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,G9340,3,10,"Encephalopathy, unspecified"
3,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,K8064,4,10,Calculus of gallbladder and bile duct with chr...
4,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,E871,5,10,Hypo-osmolality and hyponatremia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18998,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,5770,1,9,Acute pancreatitis
18999,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,30391,2,9,"Other and unspecified alcohol dependence, cont..."
19000,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,4019,3,9,Unspecified essential hypertension
19001,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694,5770,V1581,4,9,Personal history of noncompliance with medical...


In [29]:
merged_comorbidities_df.to_csv('AP_ICD_Lipase_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [30]:
# Count the number of unique patients who died in the hospital
num_patients_with_death_info = merged_comorbidities_df[merged_comorbidities_df['in_hospital_death'] == True][['subject_id', 'hadm_id']].drop_duplicates().shape[0]
print(f"Number of unique patients who died in the hospital: {num_patients_with_death_info}")

Number of unique patients who died in the hospital: 69


In [31]:
# 1. Handle missing values
missing_data = merged_comorbidities_df.isnull().sum()
print(missing_data)

subject_id           0
hadm_id              0
charttime            0
lipase_level         0
admittime            0
dischtime            0
gender               0
age                  0
race                 0
in_hospital_death    0
length_of_stay       0
icd_code_AP          0
icd_code             0
seq_num              0
icd_version          0
long_title           0
dtype: int64


In [ ]:
high_lp_filtered = high_lipase_df_unique[~high_lipase_df_unique['subject_id'].isin(ap_lp_filtered_df['subject_id'])]

In [ ]:
high_lp_filtered

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
1220,10057482,25416257,2145-04-04 15:26:00,1429.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
1221,10057482,25416257,2145-04-04 17:51:00,1390.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
1222,10057482,25416257,2145-04-05 01:00:00,1153.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
1223,10057482,25416257,2145-04-06 01:07:00,1385.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
1224,10057482,25416257,2145-04-07 02:52:00,1428.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611
...,...,...,...,...,...,...,...,...,...,...,...
13088,17509107,25893958,2122-06-07 02:05:00,999999.0,2122-06-05 00:17:00,2122-06-30 12:00:00,M,49,WHITE,False,25.488194
14505,18341278,28924376,2141-05-25 23:34:00,493.0,2141-05-25 21:27:00,2141-05-29 17:08:00,M,56,BLACK/AFRICAN AMERICAN,False,3.820139
15970,19201291,25546590,2184-01-15 02:08:00,492.0,2184-01-04 23:43:00,2184-01-22 13:15:00,M,41,BLACK/AFRICAN AMERICAN,False,17.563889
15971,19201291,25546590,2184-01-15 05:51:00,529.0,2184-01-04 23:43:00,2184-01-22 13:15:00,M,41,BLACK/AFRICAN AMERICAN,False,17.563889


In [ ]:
high_lp_filtered_nap = high_lp_filtered.groupby(['subject_id']).apply(select_row).reset_index(drop=True)
high_lp_filtered_nap['label'] = 'NON-AP'

<ipython-input-46-6ce41a92e81b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_lp_filtered_nap = high_lp_filtered.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [ ]:
high_lp_filtered_nap

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,label
0,10057482,25416257,2145-04-04 15:26:00,1429.0,2145-03-23 15:01:00,2145-04-26 17:23:00,F,76,WHITE,False,34.098611,NON-AP
1,10060142,25882608,2155-08-07 06:50:00,498.0,2155-08-06 12:40:00,2155-08-09 19:45:00,M,26,WHITE - OTHER EUROPEAN,False,3.295139,NON-AP
2,10065015,25569976,2149-03-03 15:05:00,631.0,2149-03-04 00:41:00,2149-03-15 01:43:00,F,78,WHITE,True,11.043056,NON-AP
3,10080928,25710110,2202-09-13 06:04:00,574.0,2202-09-08 04:05:00,2202-09-18 14:33:00,M,79,WHITE - OTHER EUROPEAN,False,10.436111,NON-AP
4,10095570,28612694,2148-09-28 10:40:00,1316.0,2148-09-26 03:38:00,2148-09-29 10:50:00,M,63,ASIAN - CHINESE,True,3.300000,NON-AP
...,...,...,...,...,...,...,...,...,...,...,...,...
735,19843520,25415290,2130-05-01 05:07:00,926.0,2130-04-22 04:39:00,2130-05-08 17:00:00,M,78,ASIAN - CHINESE,False,16.514583,NON-AP
736,19846637,21649404,2183-07-23 07:10:00,448.0,2183-07-23 01:12:00,2183-07-24 16:40:00,M,34,WHITE,False,1.644444,NON-AP
737,19868591,23845093,2118-04-22 02:35:00,532.0,2118-04-11 19:21:00,2118-05-01 11:06:00,M,56,BLACK/AFRICAN AMERICAN,False,19.656250,NON-AP
738,19901341,24115002,2168-03-24 06:47:00,835.0,2168-03-22 20:26:00,2168-06-13 16:56:00,F,55,WHITE,False,82.854167,NON-AP


In [ ]:
ap_lp_filtered_df['label'] = 'AP'

In [ ]:
final_dataset = pd.concat([ap_lp_filtered_df, high_lp_filtered_nap]).reset_index(drop=True)

In [ ]:
final_dataset

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay,icd_code,label
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556,K8510,AP
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167,K8580,AP
2,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944,5770,AP
3,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False,8.009028,5770,AP
4,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528,5770,AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,19843520,25415290,2130-05-01 05:07:00,926.0,2130-04-22 04:39:00,2130-05-08 17:00:00,M,78,ASIAN - CHINESE,False,16.514583,NaN,NON-AP
2102,19846637,21649404,2183-07-23 07:10:00,448.0,2183-07-23 01:12:00,2183-07-24 16:40:00,M,34,WHITE,False,1.644444,NaN,NON-AP
2103,19868591,23845093,2118-04-22 02:35:00,532.0,2118-04-11 19:21:00,2118-05-01 11:06:00,M,56,BLACK/AFRICAN AMERICAN,False,19.656250,NaN,NON-AP
2104,19901341,24115002,2168-03-24 06:47:00,835.0,2168-03-22 20:26:00,2168-06-13 16:56:00,F,55,WHITE,False,82.854167,NaN,NON-AP


In [ ]:
final_dataset.to_csv('AP_ICD_Lipase_Final_Dataset.csv', encoding='utf-8', index = False)